# Project Notebook: Optimizing DataFrames and Processing in Chunks

## 1. Introduction 

In this project, we'll practice working with chunked dataframes and optimizing a dataframe's memory usage. We'll be working with financial lending data from Lending Club, a marketplace for personal loans that matches borrowers with investors. You can read more about the marketplace on its website.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We'll be working with a dataset of loans approved from 2007-2011 (https://bit.ly/3H2XVgC). We've already removed the desc column for you to make our system run more quickly.

If we read in the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we only have 10 megabytes of memory available throughout this project, so you can practice the concepts you learned in the last two lessons.

**Tasks**

1. Read in the first five lines from `loans_2007.csv` (https://bit.ly/3H2XVgC) and look for any data quality issues.

2. Read in the first 1000 rows from the data set, and calculate the total memory usage for these rows. Increase or decrease the number of rows to converge on a memory usage under five megabytes (to stay on the conservative side).

In [2]:
# Importing pandas
import pandas as pd
pd.options.display.max_columns = 99

# Your code goes here
df_loans = pd.read_csv("loans_2007.csv")
df_loans.head()

df_loans2 = pd.read_csv("loans_2007.csv", nrows = 1000)
print("usage(MB) for 1000 rows: ",df_loans2.memory_usage(deep=True).sum()/(1024**2))


usage(MB) for 1000 rows:  1.5273666381835938


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#Increase the number of rows
df_loans = pd.read_csv("loans_2007.csv", nrows = 2000)
print(df_loans.memory_usage(deep=True).sum()/(1024**2), "MB")

3.053553581237793 MB


In [4]:
loan_iter = pd.read_csv("loans_2007.csv" , chunksize =2000)
for chunk in loan_iter:
  print(chunk.memory_usage(deep=True).sum()/(1024**2), "MB")
  

3.053553581237793 MB
3.0518617630004883 MB
3.0512466430664062 MB
3.0520782470703125 MB
3.053147315979004 MB
3.0520505905151367 MB
3.0506086349487305 MB
3.0508031845092773 MB
3.051485061645508 MB
3.050662040710449 MB
3.0520505905151367 MB
3.0516185760498047 MB
3.050469398498535 MB
3.0517873764038086 MB
3.050745964050293 MB
3.0607357025146484 MB
3.0548219680786133 MB
3.0617876052856445 MB
3.063817024230957 MB
3.1727962493896484 MB
3.1342315673828125 MB
0.868586540222168 MB


## 2. Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. In the first lesson, we explored column types by reading in the full dataframe. In this project, let's try to understand the column types better while using dataframe chunks.

**Tasks**

For each chunk:
* How many columns have a numeric type? 
* How many have a string type?
* How many unique values are there in each string column? How many of the string columns contain values that are less than 50% unique?
* Which float columns have no missing values and could be candidates for conversion to the integer type?
* Calculate the total memory usage across all of the chunks.

In [5]:
# Your code goes here
loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000)
numeric=[]
string=[]
for chunk in loan_iter:
  numeric.append(chunk.select_dtypes(include="number").shape[1])
  string.append(chunk.select_dtypes(include="object").shape[1])

print(f"Columns with numeric types: \n{numeric}")
print("\nColumns with string types: \n", string)



Columns with numeric types: 
[31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 31, 30]

Columns with string types: 
 [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 21, 22]


In [6]:
object_columns = []
loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000)
for chunk in loan_iter:
  chunk_object_columns = chunk.select_dtypes(include=["object"]).columns.tolist()
  if len(object_columns) > 0:
    similar = object_columns == chunk_object_columns
    if not similar:
      print("overall strin columns:\n", object_columns, "\n")
      print("chunk string columns:\n", chunk_object_columns, "\n")
    else:
      object_columns = chunk_object_columns
      

In [7]:
from enum import unique
#checking for strings that are less than 50%
loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000)

percentage = []
for chunk in loan_iter:
  chunk["id"] = pd.to_numeric(chunk["id"], errors = "coerce")
  chunk = chunk.dropna(axis = 0, subset = ["id"])
  unique_string_columns = chunk.select_dtypes(include=["object"]).nunique()
  count_string_columns = chunk.select_dtypes(include=["object"]).count()
  percentage.append((100*unique_string_columns/count_string_columns))
  unique_total = pd.concat(percentage)
  unique_total = unique_total.groupby(unique_total.index).mean()
  category_string_columns = list((unique_total[unique_total<50]).index)

print(print(f"\npercentage of unique values is: \n {unique_total}"))



percentage of unique values is: 
 addr_state              2.410776
application_type        0.056209
earliest_cr_line       18.636656
emp_length              0.632663
emp_title              94.211510
grade                   0.393461
home_ownership          0.193018
initial_list_status     0.056209
int_rate                3.586332
issue_d                 0.337712
last_credit_pull_d      4.069264
last_pymnt_d            3.280709
loan_status             0.194238
purpose                 0.751169
pymnt_plan              0.058481
revol_util             44.077543
sub_grade               1.937759
term                    0.097119
title                  66.508106
verification_status     0.151666
zip_code               25.880729
dtype: float64
None


In [8]:
#Float columns with no missing values

loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000)
float_count_null = []
for chunk in loan_iter:
  chunk["id"] = pd.to_numeric(chunk["id"], errors = "coerce")
  chunk = chunk.dropna(axis=0, subset = ["id"])
  float_columns_null = chunk.select_dtypes(exclude=["object"]).isnull().sum()
  float_count_null.append(float_columns_null)
  total_float_null = pd.concat(float_count_null)
  total_float_null = total_float_null.groupby(total_float_null.index).sum()
  no_missing = set((total_float_null[total_float_null == 0]).index)
  missing = set((total_float_null[total_float_null > 0]).index)

  print(f"float columns with missing values:  \n{missing}")
  print(f"\nfloat columns with no missing values:  \n{no_missing}")
  

float columns with missing values:  
set()

float columns with no missing values:  
{'total_acc', 'out_prncp', 'pub_rec_bankruptcies', 'delinq_2yrs', 'recoveries', 'total_pymnt_inv', 'total_rec_int', 'tax_liens', 'policy_code', 'acc_now_delinq', 'installment', 'dti', 'id', 'total_rec_late_fee', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'funded_amnt_inv', 'annual_inc', 'open_acc', 'revol_bal', 'chargeoff_within_12_mths', 'total_pymnt', 'inq_last_6mths', 'loan_amnt', 'total_rec_prncp', 'funded_amnt', 'pub_rec', 'last_pymnt_amnt', 'member_id', 'out_prncp_inv', 'delinq_amnt'}
float columns with missing values:  
set()

float columns with no missing values:  
{'total_acc', 'out_prncp', 'pub_rec_bankruptcies', 'delinq_2yrs', 'recoveries', 'total_pymnt_inv', 'total_rec_int', 'tax_liens', 'policy_code', 'acc_now_delinq', 'installment', 'dti', 'id', 'total_rec_late_fee', 'collection_recovery_fee', 'collections_12_mths_ex_med', 'funded_amnt_inv', 'annual_inc', 'open_acc', 'revol_b

In [9]:
#Total memroy usage

loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000)
initial_memory = []
for chunk in loan_iter:
  initial_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("total memory usage: {:.4f} MB" . format(sum(initial_memory)))



total memory usage: 65.1909 MB


## 3. Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert all of the columns where the values are less than 50% unique to the category type, and the columns that contain numeric values to the `float` type.

While working with dataframe chunks:
* Determine which string columns you can convert to a numeric type if you clean them. For example, the `int_rate` column is only a string because of the % sign at the end.
* Determine which columns have a few unique values and convert them to the category type. For example, you may want to convert the grade and `sub_grade` columns.
Based on your conclusions, perform the necessary type changes across all chunks. * Calculate the total memory footprint, and compare it with the previous one.

In [12]:
# Your code goes here
#conversion of object columns
convert_columns_dtypes = {"sub_grade":"category", "home_ownership":"category", "verification_status":"category", "purpose":"category"}

loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000, dtype = convert_columns_dtypes, parse_dates = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
total_memory=[]
for chunk in loan_iter:
  term_cleaned=chunk["term"].str.lstrip(" ").str.rstrip("months")
  int_rate_cleaned=chunk["int_rate"].str.rstrip("%")
  revol_cleaned=chunk["revol_util"].str.rstrip("%")
  chunk["term"]=pd.to_numeric(term_cleaned)
  chunk["revol_util"]=pd.to_numeric(revol_cleaned)
  chunk["int_rate"]=pd.to_numeric(int_rate_cleaned)
  total_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("\nTotal memory usage with string optimisation: {:.2f} MB" . format(sum(total_memory)))




Total memory usage with string optimisation: 38.80 MB


Obervation:
Memory usage from 65.1909MB to 38.80MB

## 4. Optimizing Numeric Columns

It looks like we were able to realize some powerful memory savings by converting to the category type and converting string columns to numeric ones.

Now let's optimize the numeric columns using the `pandas.to_numeric()` function.

**Tasks**

While working with dataframe chunks:
* Identify float columns that contain missing values, and that we can convert to a more space efficient subtype.
* Identify float columns that don't contain any missing values, and that we can convert to the integer type because they represent whole numbers.
* Based on your conclusions, perform the necessary type changes across all chunks.
* Calculate the total memory footprint and compare it with the previous one.




In [13]:
# Your code goes here

loan_iter = pd.read_csv("loans_2007.csv", chunksize = 2000, dtype = convert_columns_dtypes, parse_dates = ["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
import numpy as np
def change_to_int(df, column_name):
  max_column=df[column_name].max()
  min_column=df[column_name].min()
  for dtype_name in ["int8", "int16", "int32", "int64"]:
    if max_column < np.iinfo(dtype_name).max and min_column > np.iinfo(dtype_name).min:
      df[column_name] = df[column_name].astype(dtype_name)
      break

total_memory = []
for chunk in loan_iter:
  chunk["id"] = pd.to_numeric(chunk["id"], errors = "coerce")
  chunk = chunk.dropna(axis=0, subset = ["id"])
  term_cleaned=chunk["term"].str.lstrip(" ").str.rstrip(" months")
  int_rate_cleaned=chunk["int_rate"].str.rstrip("%")
  revol_cleaned=chunk["revol_util"].str.rstrip("%")
  chunk["term"]=pd.to_numeric(term_cleaned)
  chunk["revol_util"]=pd.to_numeric(revol_cleaned)
  chunk["int_rate"]=pd.to_numeric(int_rate_cleaned)
  float_columns=chunk.select_dtypes(include=["float"])
  float_columns=float_columns.dropna()
  for columns in float_columns.columns:
    if columns in missing:
      chunk[columns]=pd.to_numeric(chunk[columns], downcast="float")
    elif columns in no_missing:
      change_to_int(chunk, columns)
  total_memory.append(chunk.memory_usage(deep=True).sum()/(1024**2))

print("\nTotal memory usage: {:.2f} MB" . format(sum(total_memory)))
print("\nPercentage memory savings: {:.2f} %" .format(100*(sum(initial_memory) - sum(total_memory))/sum(initial_memory)))
print(f"\n {chunk.dtypes}")




Total memory usage: 33.00 MB

Percentage memory savings: 49.38 %

 id                                     int32
member_id                              int32
loan_amnt                              int16
funded_amnt                            int16
funded_amnt_inv                        int16
term                                   int64
int_rate                             float64
installment                            int16
grade                                 object
sub_grade                           category
emp_title                             object
emp_length                            object
home_ownership                      category
annual_inc                           float32
verification_status                 category
issue_d                       datetime64[ns]
loan_status                           object
pymnt_plan                            object
purpose                             category
title                                 object
zip_code                        

Observation and Conclusion:
Total memory usage = 33.00MB.
Percentage memory savings = 49.38%



## Next Steps

We've practiced optimizing a dataframe's memory footprint and working with dataframe chunks. Here's an idea for some next steps:

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

* Determine the optimal chunk size based on the memory constraints you provide.

* Determine which string columns can be converted to numeric ones by removing the `%` character.

* Determine which numeric columns can be converted to more space efficient representations.


In [ ]:
# Your code goes here